In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter  # to print to tensorboard

In [26]:
class Discriminator(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.disc = nn.Sequential(
            nn.Linear(in_features, 128),
            nn.LeakyReLU(0.01),
            nn.Linear(128, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return self.disc(x)

In [27]:
class Generator(nn.Module):
    def __init__(self, z_dim, img_dim):
        super().__init__()
        self.gen = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.LeakyReLU(0.01),
            nn.Linear(256, img_dim),
            nn.Tanh(),  # normalize inputs to [-1, 1] so make outputs [-1, 1]
        )

    def forward(self, x):
        return self.gen(x)

In [28]:
# Hyperparameters etc.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
lr = 3e-4
z_dim = 64
image_dim = 28 * 28 * 1  # 784
batch_size = 32
num_epochs = 100

cuda


In [29]:
disc = Discriminator(image_dim).to(device)
gen = Generator(z_dim, image_dim).to(device)
fixed_noise = torch.randn((batch_size, z_dim)).to(device)
transforms = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,)),
    ]
)

In [30]:
dataset = datasets.MNIST(root="./", transform=transforms, download=True)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
opt_disc = optim.Adam(disc.parameters(), lr=lr)
opt_gen = optim.Adam(gen.parameters(), lr=lr)
criterion = nn.BCELoss()
writer_fake = SummaryWriter(f"lightning_logs/fake_original")
writer_real = SummaryWriter(f"lightning_logs/real_original")
step = 0
BCE = False

In [31]:
for epoch in range(num_epochs):
    epoch_gloss = 0
    epoch_dloss = 0
    for batch_idx, (real, _) in enumerate(loader):
        real = real.view(-1, 784).to(device)
        batch_size = real.shape[0]

        ### Train Discriminator: max log(D(x)) + log(1 - D(G(z)))
        noise = torch.randn(batch_size, z_dim).to(device)
        fake = gen(noise)
        disc_real = disc(real).view(-1)
        lossD_real = criterion(disc_real, torch.ones_like(disc_real))
        disc_fake = disc(fake).view(-1)
        lossD_fake = criterion(disc_fake, torch.zeros_like(disc_fake))
        if BCE:
            lossD = (lossD_real + lossD_fake) / 2
        else:
            lossD = -(torch.mean(torch.log(disc_real) + torch.log(1. - disc_fake)))
        #writer_real.add_scalar('d_loss', lossD, epoch)
        epoch_dloss += lossD
        disc.zero_grad()
        lossD.backward(retain_graph=True)
        opt_disc.step()

        ### Train Generator: min log(1 - D(G(z))) <-> max log(D(G(z))
        # where the second option of maximizing doesn't suffer from
        # saturating gradients
        output = disc(fake).view(-1)
        if BCE:
            lossG = criterion(output, torch.ones_like(output))
        else:
            lossG = -(torch.mean(torch.log(output)))
        #writer_fake.add_scalar('g_loss', lossG, epoch)
        epoch_gloss += lossG
        gen.zero_grad()
        lossG.backward()
        opt_gen.step()

        if batch_idx == 0:
            print(
                f"Epoch [{epoch}/{num_epochs}] Batch {batch_idx}/{len(loader)} \
                      Loss D: {lossD:.4f}, loss G: {lossG:.4f}"
            )

            with torch.no_grad():
                fake = gen(fixed_noise).reshape(-1, 1, 28, 28)
                data = real.reshape(-1, 1, 28, 28)
                img_grid_fake = torchvision.utils.make_grid(fake, normalize=True)
                img_grid_real = torchvision.utils.make_grid(data, normalize=True)

                writer_fake.add_image(
                    "Mnist Fake Images", img_grid_fake, global_step=step
                )
                writer_real.add_image(
                    "Mnist Real Images", img_grid_real, global_step=step
                )
                step += 1
    print(
            f"Epoch [{epoch}/{num_epochs}] Batch {batch_idx}/{len(loader)} \
                      Loss D: {epoch_dloss/len(loader):.4f}, loss G: {epoch_gloss/len(loader):.4f}"
            )
    writer_real.add_scalar('d_loss', epoch_dloss/len(loader), epoch)
    writer_fake.add_scalar('g_loss', epoch_gloss/len(loader), epoch)
    

Epoch [0/100] Batch 0/1875                       Loss D: 1.4319, loss G: 0.6664
Epoch [0/100] Batch 1874/1875                       Loss D: 0.9079, loss G: 1.1950
Epoch [1/100] Batch 0/1875                       Loss D: 0.7784, loss G: 1.4337
Epoch [1/100] Batch 1874/1875                       Loss D: 0.9280, loss G: 1.2885
Epoch [2/100] Batch 0/1875                       Loss D: 0.7652, loss G: 1.2417
Epoch [2/100] Batch 1874/1875                       Loss D: 1.1504, loss G: 1.0705
Epoch [3/100] Batch 0/1875                       Loss D: 1.1354, loss G: 0.8769
Epoch [3/100] Batch 1874/1875                       Loss D: 1.2305, loss G: 1.0136
Epoch [4/100] Batch 0/1875                       Loss D: 0.8879, loss G: 1.3570
Epoch [4/100] Batch 1874/1875                       Loss D: 1.2678, loss G: 1.0667
Epoch [5/100] Batch 0/1875                       Loss D: 0.8079, loss G: 1.2948
Epoch [5/100] Batch 1874/1875                       Loss D: 1.3231, loss G: 1.0117
Epoch [6/100] Batch 0/